In [1]:

import numpy as np
import pandas as pd
from typing import List
# import matplotlib.pyplot as plt

from lmfit import Minimizer, minimize, Parameters, fit_report

from lib.transformations import (euler_matrix, 
                                 euler_from_matrix, 
                                 scale_from_matrix,
)
from lib.utils import convert_to_homogeneous, print_results
from lib.io import read_data_to_df


def compute_residuals(
    params: Parameters,
    x0: np.ndarray, 
    x1: np.ndarray, 
    weights: np.ndarray = None,
    prior_covariance_scale: float = None, 
) -> np.ndarray: 
    ''' 3D rototranslation with scale factor

    X1_ = T_ + m * R * X0_

    Inputs: 
    - x0 (np.ndarray): Points in the starting reference system
    - x1 (np.ndarray): Points in final reference system 
    - weights (np.ndarray, defult = None): weights (e.g., inverse of a-priori observation uncertainty)
    - prior_covariance_scale (float, default = None): A-priori sigma_0^2     
    
    Return: 
    - res (nx1 np.ndarray): Vector of the weighted residuals
        
    '''    
        
    # Get parameters  
    parvals = params.valuesdict()
    rx = parvals['rx']
    ry = parvals['ry']
    rz = parvals['rz']
    tx = parvals['tx']
    ty = parvals['ty']
    tz = parvals['tz']
    m = parvals['m']
    
    # Convert points to homogeneos coordinates and traspose np array to obtain a 4xn matrix
    x0 = convert_to_homogeneous(x0).T

    # Build 4x4 transformation matrix (T) in homogeneous coordinates
    T = np.identity(4)
    R = euler_matrix(rx, ry, rz)
    T[0:3, 0:3] = (m * np.identity(3)) @ R[:3,:3]
    T[0:3, 3:4] = np.array([tx, ty, tz]).reshape(3, 1) 
    
    # Apply transformation to x0 points
    x1_ = T @ x0
    x1_ = x1_[:3,:].T 

    # Compute residuals as differences between observed and estimated values, scaled by the a-priori observation uncertainties
    res = (x1 - x1_)
    
    # If weigthts are provided, scale residual
    if weights is not None:
        
        if weights.shape != res.shape:
            raise ValueError(
                f'Wrong dimensions of the weight matrix. It must be of size {res.shape}')
        
        res = res * weights
    
    return res.flatten()

In [23]:
# MAIN
pt_loc = read_data_to_df('../data/rotra3d/points3d_loc.txt',
                         header=None,
                         index_col=0
                         )
pt_world = read_data_to_df('../data/rotra3d/points3d_world.txt',
                         header=None,
                         index_col=0
                        )
print(f'Points loc: \n {pt_loc}')
# print(f'Points world: \n {pt_world}')

t_ini = np.array(
    [1.46882746e+02, 8.74147624e+01, 9.04722323e+01], 
    dtype='float64'
)
rot_ini = np.array((-1.455234490428092, 0.06619166269889347,
                   0.9470055218154193), 'float64')
m_ini = float(1.0)

# Define Parameters to be optimized
params = Parameters()
params.add('rx', value=rot_ini[0], vary=True)
params.add('ry', value=rot_ini[1], vary=True)
params.add('rz', value=rot_ini[2], vary=True)
params.add('tx', value=t_ini[0], vary=True)
params.add('ty', value=t_ini[1], vary=True)
params.add('tz', value=t_ini[2], vary=True)
params.add('m',  value=m_ini, vary=True)

# Define Weights as the inverse of the a-priori standard deviation of each observation
# All the measurements are assumed as independent (Q diagonal)
# Weight matrix must have the same shape as the observation matrix (or vector) X0
uncertainty = 1e-2 * np.array( #Values are given in [cm] and converted in [m]
    [
        [5., 5., 5.],
        [5., 5., 5.],
        [1., 1., 1.],
        [1., 1., 1.],
        [10., 10., 10.],
        [5., 5., 5.],
        [5., 5., 5.],        
        [5., 5., 5.],        
        [5., 5., 5.],        
        [5., 5., 5.],        
        [5., 5., 5.],        
        [5., 5., 5.],        
        [5., 5., 5.],        
        
    ]
)
weights = 1. / uncertainty

# A-priori Sigma_0²: scale of the covariance matrix
sigma0_2 = 1

# # Remove observations
# obs_to_remove = []
# for i in obs_to_remove:
#     pt_loc = np.delete(pt_loc, (i), axis=0)
#     pt_world = np.delete(pt_world, (i), axis=0)
#     weights = np.delete(weights, (i), axis=0)
# rows_to_drop = ['T2', 'T4']
# pt_loc = pt_loc.drop(rows_to_drop)
# pt_utm = pt_world.drop(rows_to_drop)
# obs_to_remove = [2,1]
# for i in obs_to_remove:
#     weights = np.delete(weights, (i), axis=0)

# Run Optimization!
minimizer = Minimizer(
    compute_residuals,
    params,
    fcn_args=(
        pt_loc.to_numpy(),
        pt_world.to_numpy(),
    ),
    fcn_kws={
        'weights': weights,
        'prior_covariance_scale': sigma0_2,
    },
    scale_covar=True,
)
result = minimizer.minimize(method='leastsq')


# Print result
print_results(result, weights, sigma0_2)


Points loc: 
              1          2           3
0                                    
T2 -118.003740 -53.321677  647.109548
T4 -164.581465 -27.793899  653.455977
F2   19.736779  33.287878  132.161881
F3   78.606606  -2.223028  254.460008
F4   80.539316  14.790648  186.589377
F5   -2.340132  26.345179  113.815316
11  -43.110640 -10.236377  374.378606
12   15.134323  -0.025228  246.939683
13    6.696014  19.638145  208.905036
15   16.408720  17.880793  212.910331
16   38.347625  29.963605  194.078566
17   60.718382  14.786779  229.541008
19   14.290375  -8.854948  354.189050
-------------------------------
Optimization report
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 33
    # data points      = 39
    # variables        = 7
    chi-square         = 7487.55760
    reduced chi-square = 233.986175
    Akaike info crit   = 219.040015
    Bayesian info crit = 230.684947
[[Variables]]
    rx: -1.46698112 +/- 0.00877062 (0.60%) (init = -1.455234)
    ry:  

In [20]:
ndim = 3
residuals = result.residual.reshape(-1, ndim)
print(residuals)

residuals = residuals / weights

for res in residuals:
    for dim in range(ndim):
        if dim == ndim-1:
            endline = '\n'
        else:
            endline = ' '
        print(f"{res[dim]:8.3f}", end=endline)


[[-278.9920072   142.22892435   91.81748927]
 [-372.46115581  168.00693405  110.65875979]
 [ -58.05361279  381.31339884  117.05519992]
 [ 107.99913343 -459.62305992 -143.98126716]
 [ -13.44479394  -32.89396804   -1.55167179]
 [  -5.98341645  129.27137294    8.05982032]
 [ 111.64440146    9.61516092  -21.42451917]
 [  70.90382067   -2.11033441  -31.78789915]
 [  56.82267953   35.50620949   -3.94864516]
 [  48.22673598   19.01992603   -7.86613369]
 [  21.78306078   -3.15896242   17.26306237]
 [  26.16832697  -55.36141963   -3.79663365]
 [  78.88234806  -35.02246453  -23.56912885]]
 -13.950    7.111    4.591
 -18.623    8.400    5.533
  -0.581    3.813    1.171
   1.080   -4.596   -1.440
  -1.344   -3.289   -0.155
  -0.299    6.464    0.403
   5.582    0.481   -1.071
   3.545   -0.106   -1.589
   2.841    1.775   -0.197
   2.411    0.951   -0.393
   1.089   -0.158    0.863
   1.308   -2.768   -0.190
   3.944   -1.751   -1.178
